In [ ]:
import torch, torchtext, numpy as np
import pandas as pd, csv
from torch import nn, optim
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import pdb
torch.manual_seed(291)
np.random.seed(291)
import pandas as pd

In [ ]:
!wget https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/ratings.csv
!wget https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/books.csv

--2021-05-16 18:01:21--  https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/ratings.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 72126826 (69M) [text/plain]
Saving to: ‘ratings.csv’

ratings.csv         100%[===================>]  68.79M   225MB/s    in 0.3s    

2021-05-16 18:01:22 (225 MB/s) - ‘ratings.csv’ saved [72126826/72126826]

--2021-05-16 18:01:22--  https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/books.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3286659 (3.1M) [text/plain]
Sav

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, fn):
        self.dataframe = pd.read_csv(fn)
        u2n = { u: n for n, u in enumerate(self.dataframe['user_id'].unique()) } 
        apply_conv = { m: conv_arr[m] for m in self.dataframe["book_id"] }
        self.dataframe['user_id'] = self.dataframe['user_id'].apply(lambda u: u2n[u])
        self.dataframe["book_id"] = self.dataframe["book_id"].apply(lambda m: apply_conv[m])
        self.dataframe = self.dataframe[self.dataframe["book_id"] != -1]
        self.coords = torch.LongTensor(self.dataframe[['user_id', 'book_id']].values)
        self.ratings = torch.FloatTensor(self.dataframe['rating'].values)
        self.n_users = self.dataframe['user_id'].nunique()
        self.n_boooks = self.dataframe['book_id'].nunique()
    
    def __len__(self):
        return len(self.coords)

    def __getitem__(self, i):
        return (self.coords[i], self.ratings[i])         

In [ ]:
bookFP = "books.csv"
ratingFP = "ratings.csv"

books = pd.read_csv(bookFP)
#Compile all non-english languages - indices determined by inspection
foreign = books["language_code"].unique()[6:].tolist()
foreign.append(books["language_code"].unique()[4])
#iterate through foreign languages keeping only books that aren't in that language
for lang in foreign:
    books = books[books["language_code"] != lang]
books = books[books["title"] != "Mein Kampf"] #remove this book, we do not under any circumstances want to recommend it
count = 0
conv_arr = [-1] * 10001
for entry in books['book_id']:
  conv_arr[entry] = count
  count = count + 1
book_conv = {m : conv_arr[m] for m in books['book_id']}
books['book_id'] = books['book_id'].apply(lambda m: book_conv[m]) 

In [ ]:
ds_full = Dataset('ratings.csv')

In [ ]:
n_train = int(0.72 * len(ds_full))
n_test = int(0.18 * len(ds_full))
n_RL = len(ds_full) - n_train - n_test
rng = torch.Generator().manual_seed(291)
ds_train, ds_test, ds_RL = torch.utils.data.random_split(ds_full, [n_train, n_test, n_RL], rng)
len(ds_full)

5914830

In [ ]:
#Code adapted and inspired by movie recommender model in lecture 6
class BookRecommenderEmbeddingML(nn.Module):
  def __init__(self, n_users, n_books, emb_dim):
    super(BookRecommenderEmbeddingML, self).__init__()
    self.user_embedding = nn.Embedding(n_users, emb_dim)
    self.book_embedding = nn.Embedding(n_books, emb_dim)
    nn.init.xavier_uniform_(self.user_embedding.weight)
    nn.init.xavier_uniform_(self.book_embedding.weight)
    self.dropout = nn.Dropout(0.25)
  
  def forward(self, samples):
    users = self.user_embedding(samples[:,0]) # gets embedding of users
    users = self.dropout(users)
    books = self.book_embedding(samples[:,1])
    books = self.dropout(books)
    return torch.sigmoid(dot) * 5.5    

In [ ]:
device = torch.device('cuda:0')

def run_test(model, ldr, crit):
    total_loss, total_count = 0, 0
    model.eval()
    tq_iters = tqdm(ldr, leave=False, desc='test iter')
    with torch.no_grad():
        for coords, labels in tq_iters:
            coords, labels = coords.to(device), labels.to(device)
            preds = model(coords)
            loss = crit(preds, labels)
            total_loss += loss.item() * labels.size(0)
            total_count += labels.size(0)
            tq_iters.set_postfix({'loss': total_loss/total_count}, refresh=True)
    return total_loss / total_count

def run_train(model, ldr, crit, opt, sched):
    model.train()
    total_loss, total_count = 0, 0
    tq_iters = tqdm(ldr, leave=False, desc='train iter')
    for (coords, labels) in tq_iters:
        opt.zero_grad()
        coords, labels = coords.to(device), labels.to(device)
        preds = model(coords)
        loss = crit(preds, labels)
        loss.backward()
        opt.step()
        sched.step()
        total_loss += loss.item() * labels.size(0)
        total_count += labels.size(0)
        tq_iters.set_postfix({'loss': total_loss/total_count}, refresh=True)
    return total_loss / total_count

def run_all(model, ldr_train, ldr_test, crit, opt, sched, n_epochs=10):
    best_loss = np.inf
    tq_epochs = tqdm(range(n_epochs), desc='epochs', unit='ep')
    for epoch in tq_epochs:
        train_loss = run_train(model, ldr_train, crit, opt, sched)
        test_loss = run_test(model, ldr_test, crit)
        tqdm.write(f'epoch {epoch}   train loss {train_loss:.6f}    test loss {test_loss:.6f}')
        if test_loss < best_loss:
            best_loss = test_loss
            tq_epochs.set_postfix({'bE': epoch, 'bL': best_loss}, refresh=True)

In [ ]:
model0_H = BookRecommenderEmbeddingML(ds_full.n_users, ds_full.n_books, 24)
model0_H.dropout = nn.Dropout(0.35)
model0_H.to(device)
ldr_train = torch.utils.data.DataLoader(ds_train, batch_size=32, shuffle=True)
ldr_test = torch.utils.data.DataLoader(ds_test, batch_size=32)

crit = nn.MSELoss().to(device)
opt = optim.SGD(model0_H.parameters(), lr=1e-6,momentum=0.75)
sched = optim.lr_scheduler.OneCycleLR(opt, max_lr=0.1, steps_per_epoch=len(ldr_train), epochs= 15)

run_all(model0_H, ldr_train, ldr_test, crit, opt, sched, 15)

epoch 0   train loss 2.314258    test loss 1.880845


epoch 1   train loss 1.132340    test loss 0.775775


epoch 2   train loss 0.875577    test loss 0.753936


epoch 3   train loss 0.876222    test loss 0.748865


epoch 4   train loss 0.879805    test loss 0.745725


...

epoch 10   train loss 0.816929    test loss 0.710447


epoch 11   train loss 0.792406    test loss 0.698854


epoch 12   train loss 0.767186    test loss 0.689500


epoch 13   train loss 0.744287    test loss 0.683986


epoch 14   train loss 0.730798    test loss 0.682919



In [ ]:
model1_H = BookRecommenderEmbeddingML(ds_full.n_users, ds_full.n_books, 24)
model1_H.dropout = nn.Dropout(0.45)
model1_H.to(device)
ldr_train = torch.utils.data.DataLoader(ds_train, batch_size=32, shuffle=True)
ldr_test = torch.utils.data.DataLoader(ds_test, batch_size=32)

crit = nn.MSELoss().to(device)
opt = optim.SGD(model1_H.parameters(), lr=1e-6,momentum=0.75)
sched = optim.lr_scheduler.OneCycleLR(opt, max_lr=0.1, steps_per_epoch=len(ldr_train), epochs= 15)

run_all(model1_H, ldr_train, ldr_test, crit, opt, sched, 15)

epoch 0   train loss 2.317479    test loss 1.906875


epoch 1   train loss 1.201447    test loss 0.784456


epoch 2   train loss 0.965134    test loss 0.770425


epoch 3   train loss 0.986746    test loss 0.774309


epoch 4   train loss 1.002882    test loss 0.775339


...

epoch 10   train loss 0.919854    test loss 0.740781


epoch 11   train loss 0.888942    test loss 0.728484


epoch 12   train loss 0.857562    test loss 0.718432


epoch 13   train loss 0.831136    test loss 0.713066


epoch 14   train loss 0.816383    test loss 0.711903



In [ ]:
model2_H = BookRecommenderEmbeddingML(ds_full.n_users, ds_full.n_books, 24)
model2_H.dropout = nn.Dropout(0.25)
model2_H.to(device)
ldr_train = torch.utils.data.DataLoader(ds_train, batch_size=32, shuffle=True)
ldr_test = torch.utils.data.DataLoader(ds_test, batch_size=32)

crit = nn.MSELoss().to(device)
opt = optim.SGD(model2_H.parameters(), lr=1e-6,momentum=0.75)
sched = optim.lr_scheduler.OneCycleLR(opt, max_lr=0.1, steps_per_epoch=len(ldr_train), epochs= 15)

run_all(model2_H, ldr_train, ldr_test, crit, opt, sched, 15)

epoch 0   train loss 2.314031    test loss 1.878144


epoch 1   train loss 1.089132    test loss 0.768444


epoch 2   train loss 0.816930    test loss 0.739329


epoch 3   train loss 0.799844    test loss 0.730385


epoch 4   train loss 0.791284    test loss 0.725557


...

epoch 10   train loss 0.720100    test loss 0.694669


epoch 11   train loss 0.697133    test loss 0.684639


epoch 12   train loss 0.673155    test loss 0.676290


epoch 13   train loss 0.650135    test loss 0.671583


epoch 14   train loss 0.635701    test loss 0.670651



In [ ]:
model3_H = BookRecommenderEmbeddingML(ds_full.n_users, ds_full.n_books, 24)
model3_H.dropout = nn.Dropout(0.25)
model3_H.to(device)
ldr_train = torch.utils.data.DataLoader(ds_train, batch_size=32, shuffle=True)
ldr_test = torch.utils.data.DataLoader(ds_test, batch_size=32)

crit = nn.MSELoss().to(device)
opt = optim.SGD(model3_H.parameters(), lr=1e-6,momentum=0.75)
sched = optim.lr_scheduler.OneCycleLR(opt, max_lr=0.1, steps_per_epoch=len(ldr_train), epochs= 30)

run_all(model3_H, ldr_train, ldr_test, crit, opt, sched, 30)

epoch 0   train loss 2.348873    test loss 2.333250


epoch 1   train loss 1.574311    test loss 0.915281


epoch 2   train loss 0.857247    test loss 0.745452


epoch 3   train loss 0.793362    test loss 0.727381


epoch 4   train loss 0.780166    test loss 0.720405


...

epoch 16   train loss 0.746222    test loss 0.712549


epoch 17   train loss 0.739164    test loss 0.710717


epoch 18   train loss 0.731060    test loss 0.708000


epoch 19   train loss 0.723015    test loss 0.703860


epoch 20   train loss 0.711568    test loss 0.697618


In [ ]:
model4_H = BookRecommenderEmbeddingML(ds_full.n_users, ds_full.n_boooks, 24)
model4_H.dropout = nn.Dropout(0.25)
model4_H.to(device)
ldr_train = torch.utils.data.DataLoader(ds_train, batch_size=32, shuffle=True)
ldr_test = torch.utils.data.DataLoader(ds_test, batch_size=32)

crit = nn.MSELoss().to(device)
opt = optim.SGD(model4_H.parameters(), lr=1e-6,momentum=0.75)
sched = optim.lr_scheduler.OneCycleLR(opt, max_lr=0.1, steps_per_epoch=len(ldr_train), epochs= 20)

run_all(model4_H, ldr_train, ldr_test, crit, opt, sched, 20)

epoch 0   train loss 2.342901    test loss 2.183546


epoch 1   train loss 1.249887    test loss 0.796388


epoch 2   train loss 0.816887    test loss 0.741477


epoch 3   train loss 0.794806    test loss 0.730986


epoch 4   train loss 0.788814    test loss 0.727244


...

epoch 15   train loss 0.686609    test loss 0.683423


epoch 16   train loss 0.666800    test loss 0.677173


epoch 17   train loss 0.648586    test loss 0.671852


epoch 18   train loss 0.633146    test loss 0.669418


epoch 19   train loss 0.624227    test loss 0.668937



##Modified Dataset to Extract Bad Book

In [ ]:
model5_H = BookRecommenderEmbeddingML(ds_full.n_users, ds_full.n_boooks, 24)
model5_H.dropout = nn.Dropout(0.25)
model5_H.to(device)
ldr_train = torch.utils.data.DataLoader(ds_train, batch_size=32, shuffle=True)
ldr_test = torch.utils.data.DataLoader(ds_test, batch_size=32)

crit = nn.MSELoss().to(device)
opt = optim.SGD(model5_H.parameters(), lr=1e-6,momentum=0.75)
sched = optim.lr_scheduler.OneCycleLR(opt, max_lr=0.1, steps_per_epoch=len(ldr_train), epochs= 20)

run_all(model5_H, ldr_train, ldr_test, crit, opt, sched, 20)

epoch 0   train loss 2.344299    test loss 2.221309


epoch 1   train loss 1.285630    test loss 0.800045


epoch 2   train loss 0.825506    test loss 0.740234


epoch 3   train loss 0.797331    test loss 0.728270


epoch 4   train loss 0.788562    test loss 0.723579


...

epoch 15   train loss 0.686029    test loss 0.682467


epoch 16   train loss 0.666440    test loss 0.675712


epoch 17   train loss 0.647582    test loss 0.670571


epoch 18   train loss 0.632887    test loss 0.667879


epoch 19   train loss 0.623315    test loss 0.667441



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
torch.save({
            'full' : ds_full,
            'train': ds_train,
            'test': ds_test,
            # 'RL': ds_RL
            
            }, '/content/drive/My Drive/datasets-for-new-best-1.pt')

In [ ]:
torch.save({
            'best-model' : model5_H 
}, '/content/drive/My Drive/new-best-1.pt')

##Citations
Below are citations for the use of the datasets used in the code above  
(Citations are in the style specified by the dataset compiler)  

@article{goodbooks2017,
    author = {Zajac, Zygmunt},
    title = {Goodbooks-10k: a new dataset for book recommendations},
    year = {2017},
    publisher = {FastML},
    journal = {FastML},
    howpublished = {\url{http://fastml.com/goodbooks-10k}},
}